<a href="https://colab.research.google.com/github/Videothek/machine-learning/blob/main/Agentensystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF pytesseract Pillow langchain-community langchain_openai docarray faiss-cpu
!apt-get update
!apt-get install tesseract-ocr

## Definieren relevanter Variablen

In [ ]:
# Name des Google-Colab-Secret für den OpenAI-Key
colab_secret = "oai_apikey"

# Name des Google-Colab-Secret für den OpenWeather-Key
ow_colab_secret = "ow_apikey"

# OpenWeather URL um die Wetterdaten abzurufen
WEATHER_BASE_URL= "https://api.openweathermap.org/data/2.5/weather?"

# Festlegen der URL von der das PDF-Dokument geladen werden kann.
url = "https://raw.githubusercontent.com/Videothek/machine-learning/main/MCP.pdf"

# Embedding Model für das Chunking festlegen.
embedding_model="text-embedding-3-small"

# Eingabe der Chunk Größe (Token/Chunk)
input_chunk_size = 300

# Eingabe des Overlaps (Festlegen der Informationslücke)
input_overlap = 50

# Large Language Model für die Generierung der Antwort festlegen.
large_language_model = "gpt-4o-mini"

# Eingabe der Temperatur (Kreativität) der Antwort
input_temperature = 0.3

# Eingabe der maximalen Output-Token des LLM
max_output_tokens = 300

In [ ]:
# Importieren des Pakets für den Zugriff auf den Google-Colab-KeyStore.
from google.colab import userdata

# Auslesen des OpenAI-Key aus dem Google-Colab-KeyStore.
OPENAI_API_KEY = userdata.get(colab_secret)

# Auslesen des OpenWeather-Key aus dem Google-Colab-Keystore
WEATHER_API_KEY= userdata.get(ow_colab_secret)

## Vorbereiten des PDF-Dokument für die Nutzung durch das RAG-Tool

In [ ]:
# Importieren der notwendigen Pakete für den Abruf des PDF-Dokument.
import requests
import io

# Herunterladen der pdf Datei als Kontext für das RAG.
response = requests.get(url)

# Speichern des PDF-Byte-Stream.
pdf_file = io.BytesIO(response.content)

print(f"Status Code: {response.status_code}")

In [ ]:
# Importieren von PyMuPDF um das PDF in Bilder zu verwandeln.
import fitz

# Array für die PDF-Bilder initialisieren.
pdf_images = []

# PDF-Datei öffnen.
doc = fitz.open(stream=pdf_file, filetype="pdf")

# Über die Seiten des PDF-Dokument iterieren.
for page_num in range(len(doc)):

    # Seite des PDF-Dokument laden.
    page = doc.load_page(page_num)

    # PDF-Dokument als pixmap laden.
    pix = page.get_pixmap(dpi=300)

    # Pixmap in PNG konvertieren.
    img_bytes = pix.tobytes(output="png")

    # Seite des PDF-Dokument zum Array hinzufügen.
    pdf_images.append(img_bytes)

# PDF-Dokument schließen.
doc.close()

print(f"Seiten des PDF-Dokumentes: {len(pdf_images)}")

In [ ]:
# Importieren der Pakete um Text aus den PDF-Bildern auszulesen.
import pytesseract
from PIL import Image

# Array für Textinhalte des PDF-Dokument initialisieren.
extracted_text = []

# Durch die Bilder des PDF-Dokument iterrieren.
for image_bytes in pdf_images:

    # Bild des PDF-Dokument öffnen.
    image = Image.open(io.BytesIO(image_bytes))

    # Text aus dem Bild extrahieren.
    text = pytesseract.image_to_string(image)

    # Text aus dem Bild zum Array hinzufügen.
    extracted_text.append(text)

print(f"Seiten mit erkanntem Text: {len(extracted_text)}")

In [ ]:
# Variable für den Text definieren.
pdf_document = ""

# Durch den erkannten Text iterrieren.
for text in extracted_text:

    # Text zusammenfügen.
    pdf_document += text

print(f"Anzahl der erkannten Zeichen: {len(pdf_document)}")

In [ ]:
# Importieren des Pakets um den Text aus dem PDF-Dokument in Chunks zu zerlegen.
import tiktoken

# Encoding für das genutzte Embedding-Modell abrufen,
# dies hilft die Tokenanzahl zu kontrollieren und Kosten zu optimieren.
encoding = tiktoken.encoding_for_model(embedding_model)

# Das Dokument wird in Tokens zerlegt, welche die kleinste Einheit sind, die das LLM verarbeitet.
# Das zerlegen in Token folgt dem speziellen Encoding für das jeweilige Embedding-Modell.
# Die Token werden in diesem Fall benötigt, um das PDF-Dokument in Chunks zu zerlegen
tokens = encoding.encode(pdf_document)

# Diese Variable gibt an, wie viele Token maximal in jedem Chunk des PDF-Dokument sein dürfen.
chunk_size = input_chunk_size

# Diese Variable gibt an, wie viele Token in einem Chunk überlappen, um den Kontext besser zu erhalten.
# Dadurch werden Informationslücken am Chunkrand vermieden.
overlap = input_overlap

# Array für die Chunks initialisieren.
chunks = []

# Kontrollvariable für die while-Schleife initialisieren.
start = 0

# Alle erstellten Tokens durchlaufen, bis alle Token einem Chunk zugeordnet wurden.
while start < len(tokens):

    # Speichern der für die chunk_size festgelegten Anzahl an Token in einen Chunk.
    chunk = tokens[start:start + chunk_size]

    # Den neuen Chunk zum Array aller Chunks hinzufügen.
    chunks.append(encoding.decode(chunk))

    # Bei den Token um die festgelegte overlap Anzahl zurückgehen, um den overlap zu berücksichtigen
    # und Informationsverlust zu vermeiden.
    start += chunk_size - overlap

print(f"Anzahl der Token: {len(tokens)}")
print(f"Anzahl der Chunks: {len(chunks)}")

In [ ]:
# Importieren der Pakete um das Embedding-Modell aufzurufen, die Chunks-Liste in eine Documents-Liste zu verwandeln
# und die Embeddings in einer Vektordatenbank zu speichern.
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

# Festlegen des Embedding-Modells, für die spätere Nutzung bei dem Embedding der Chunks.
embeddings = OpenAIEmbeddings(

    # Festlegen des Embedding-Modells.
    model=embedding_model,

    # Festlegen des API_KEY für OpenAI.
    openai_api_key=OPENAI_API_KEY
)

# Verwandeln der Chunks-Liste in eine Documents-Liste, um diese als einen FAISS-Vektor speichern zu können.
chunks_document = [Document(page_content=chunk) for chunk in chunks]

# Embedding der Chunks und Speicherung in einem FAISS-Index für die Nutzung in der Chain.
# In diesem Schritt werden die Chunks in das Embedding-Modell gegeben, welches die Chunks bzw. die Token
# in Word-embeddings verwandelt, um diese für das LLM abrufbar zu machen.
# FAISS steht für Facebook AI Similarity Search und ist eine Open-Source Bibliothek für die Vektorsuche,
# es berechnet später in der Chain die Ähnlichkeit der Frage zu den gespeicherten Embeddings der Chunks
# und gibt anschließend die relevantesten Chunks an das LLM weiter, welches basierend darauf die Antwort generiert.
vectorstore = FAISS.from_documents(chunks_document, embeddings)

## Implementierung der Tools die der Agent nutzen kann

In [ ]:
# Importieren des LangChain Pakets um die Tools für den Agenten zu definieren.
from langchain.tools import tool

# Definieren des ersten Tools: RAG-System.
@tool
def rag_tool(query: str) -> str:

    # Beschreibung auf dessen Basis der Agent selbstständig entscheidet, bei welcher Art von Benutzereingabe das Tool verwendet werden soll.
    """Searches for relevant information in the loaded document based on the query."""

    # Auslesen der Top-3 zur Benutzereingabe passenden Chunks aus dem PDF-Dokument.
    chunks = vectorstore.similarity_search(query, k=3)

    # Zusammenfügen der Chunks für die mitgabe im Kontext.
    context = "\n".join([c.page_content for c in chunks])

    # Zusammenbauen des Kontext für die Antwort des LLM bei der Ausgabe.
    context = f"Beantworte die Frage ausschließlich basierend auf folgendem Kontext:\n{context}\nFrage: {query}. Wenn Du die Frage nicht beantworten kannst, antworte: Ich weiß es nicht"

    # Zurückgeben des Kontext, damit der Agent diesen an das LLM weitergeben kann.
    return context

# Definieren des zweiten Tools: Abfrage eine Wetter-API
@tool
def get_current_weather(location: str) -> str:

    # Beschreibung auf dessen Basis der Agent selbstständig entscheidet, bei welcher Art von Benutzereingabe das Tool verwendet werden soll.
    """Fetches the current weather information for a given location."""

    # Zusammenbauen der URL für den Abruf der OpenWeather-API.
    complete_url = WEATHER_BASE_URL + "appid=" + WEATHER_API_KEY + "&q=" + location + "&units=metric"

    # Abfrage der OpenWeather-API.
    response = requests.get(complete_url)

    # Überprüfung ob die Abfrage der API erfolgreich war.
    if response.status_code != 200:

        # Rückgabe eines Fehler an das LLM, damit dieses eine passende Ausgabe generieren kann.
        return f"Error fetching weather data for {location}: {weather_data['message']}"

    # Speichern der API-Antwort im JSON-Format als Python-Objekt.
    weather_data = response.json()

    # Auslesen der Wetterdaten aus der API-Antwort.
    main_data = weather_data["main"]
    temperature = main_data["temp"]
    humidity = main_data["humidity"]
    description = weather_data["weather"][0]["description"]

    # Zusammenbauen des Kontext für die Antwort des LLM bei der Ausgabe.
    context = f"Das Wetter in {location}: Temperatur - {temperature}°C, Luftfeuchtigkeit - {humidity}%, Beschreibung - {description}."

    # Zurückgeben des Kontext, damit der Agent diesen an das LLM weitergeben kann.
    return context

## Implementierung einer unstrukturierten Antwort des Agenten

In [ ]:
# Importieren der LangChain Pakete, um den Agenten zu bauen und die Eingabe zu formatieren.
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnableLambda, RunnableSequence

# Festlegen des LLM für den Agenten, welcher selbstständig das passende Tool auswählt
# und eine passende Antwort, basierend auf den durch das jeweilige Tool bereitgestellte Kontext, generiert.
model = ChatOpenAI(

    # Festlegen des Large-Language-Modells.
    model=large_language_model,

    # Festlegen des API_KEY für OpenAI.
    openai_api_key=OPENAI_API_KEY,

    # Festlegen der Kreativität der Antworten.
    # 0.3 liefert nicht immer die gleiche Antwort, aber ist faktenbasiert.
    temperature=input_temperature,

    # Maximale Anzahl der Ausgabetoken des Large Language Model, um die Kosten unter Kontrolle zu halten.
    max_tokens=max_output_tokens
)

# Festlegen des Prompt, welcher an den Agenten übergeben wird, und eine Anweisung erhält, was der Agent tun soll.
# Zudem wird die Eingabe des Benutzers entsprechend eingefügt.
# Wichtig für Agentensysteme ist zudem den Gedankenprozess für eine Entscheidungsfindung in den Prompt einzubeten,
# was im Falle der LangChain Prompt über {agent_scratchpad} passiert.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assitent, der das aktuelle Wetter für einen Standort abruft, oder Informationen aus einem PDF-Dokument beantwortet."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Speichern der verfügbaren Tools in einer Liste, für die Übergabe an den Agenten.
tools = [rag_tool,get_current_weather]

# Initialisieren des Agent, der das model 'model' nutzt und auf die Tools 'tools' zugreifen kann und den Prompt 'prompt'
# als Eingabe entgegennimmt um eine Tool-Entscheidung zu treffen und eine Antwort zu generieren.
agent = create_tool_calling_agent(model, tools, prompt)

# Erstellen eines AgentExecuter, der die Schritte des Agenten sowie die Eingabe, Tool-Nutzung und Ausgabe koordiniert.
agent_unstructured = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Definieren einer LangChain lambda, um diese in der Chain verwenden zu können.
format_input = RunnableLambda(

    # Formatieren der Eingabe durch eine lambda für das Einfügen in den Prompt.
    lambda user_input: {"input": user_input}
)

# Definieren der LangChain Chain, welche zuerst die Eingabe formatiert, diese in die Prompt einfügt
# und anschließend den Agenten ausführt, welcher anhand der Prompt die Fragen beantwortet.
chain = format_input | agent_unstructured

# Anfrage an den Agent mit einer Frage, die er durch das PDF-Dokument aus dem RAG-System beantworten kann.
chain.invoke("Was ist wichtig um effektiv mit MCP arbeiten zu können?")

# Anfrage an den Agenten, wie das Wetter in Stuttgart ist, was er durch die Abfrage der OpenWaether-API beantworten kann.
chain.invoke("Wie ist das Wetter in Stuttgart?")

# Anfrage an den Agent, die er durch keins der Tools beantworten kann, und daher auf sein atrainiertes Wissen zurückgreifen muss.
chain.invoke("Wer ist der aktuelle US-Präsident?")

## Implementierung einer strukturierten Antwort des Agenten

In [ ]:
# Importieren der Pakete um die Formatierungsanweisungen für die Ausgabe an den Agenten zu übergeben.
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Definieren eines Datenmodells das angibt, welcher Felder das JSON-Objekt haben soll und welche Antworten jeweils zugeordnet werden.
class WeatherInfo(BaseModel):
    location: str = Field(description="Der Name der Stadt")
    temperature: float = Field(description="Die aktuelle Temperatur in Grad Celsius")
    humidity: float = Field(description="Die aktuelle Luftfeuchtigkeit in Prozent")
    description: str = Field(description="Eine kurze Beschreibung der Wetterlage")

# Definieren des Ausgabe-Parser, um dem LLM mitzuteilen, wie es die Daten in der Antwort ausgeben soll.
output_parser = PydanticOutputParser(pydantic_object=WeatherInfo)

# Definieren der neuen Prompt, mit dem Platzhalter für die Formatierungsanweisungen.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assitent, der das aktuelle Wetter für einen Standort abruft, oder Informationen aus einem PDF-Dokument beantwortet. Antworte nur mit den benötigten Informationen und formatieren die Ausgabe als JSON basierend auf der folgenden Anweisung:\n{format_instructions}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Einfügen der Formatierungsanweisungen in den Prompt, um diese nicht bei jedem Aufruf mitgeben zu müssen.
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

# Initialisieren des Agent, der das model 'model' nutzt und auf die Tools 'tools' zugreifen kann und den Prompt 'prompt'
# als Eingabe entgegennimmt um eine Tool-Entscheidung zu treffen und eine Antwort zu generieren.
agent = create_tool_calling_agent(model, tools, prompt)

# Erstellen eines AgentExecuter, der die Schritte des Agenten sowie die Eingabe, Tool-Nutzung und Ausgabe koordiniert.
agent_structured = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Definieren einer LangChain lambda, um diese in der Chain verwenden zu können.
format_input = RunnableLambda(

    # Formatieren der Eingabe durch eine lambda für das Einfügen in den Prompt.
    lambda user_input: {"input": user_input}
)

# Definieren einer LangChain lambda, um diese für das filtern des JSON-Objekt aus der Antwort des LLM verwenden zu können.
extract_output = RunnableLambda(

    # Extrahieren des JSON-Objekt aus der Antwort des LLM durch eine lambda.
    lambda x: x.get('output', x)
)

# Definieren der LangChain Chain, welche zuerst die Eingabe formatiert, diese in die Prompt einfügt, den Agenten ausführt,
# welcher anhand der Prompt das Wetter abruft und als strukturiertes JSON-Objekt zurückgibt, welches im letzten Schritt
# aus der Antwort des LLM extrahiert wird.
chain = format_input | agent_structured | extract_output

# Anfrage an den Agent, wie das Wetter in Stuttgart ist, was er durch die Abfrage der OpenWaether-API beantworten kann.
# Durch den Kontext aus der Prompt weiß er zudem, dass beim nennen einer Stadt, die Wetterinformationen abgerufen werden müssen.
output = chain.invoke("Stuttgart")

# Ausgabe des formatierten JSON-Objekts mit den entsprechenden Werten.
print(f"Antwort des LLM im JSON-Format:\n{output}")

# Anfrage an den Agent mit einer Frage, die er durch das PDF-Dokument aus dem RAG-System beantworten kann.
# Hier liefert der Agent ebenfalls eine Antwort in Form eines JSON-Objekts zurück, jedoch nicht im beschriebenen Format,
# da er selbsständig festgestellt hat, dass dieses Format nicht auf die Daten passt.
# Stattdessen hat sich das LLM auf Basis seiner antrainierten Daten eine eigene Formatierung überlegt und die Ausgabe entsprechend formatiert.
output = chain.invoke("Was ist wichtig um effektiv mit MCP arbeiten zu können?")

# Ausgabe des formatierten JSON-Objekts mit den entsprechenden Werten.
print(f"Antwort des LLM im JSON-Format:\n{output}")

# Anfrage an den Agent, die er durch keins der Tools beantworten kann, und daher auf sein atrainiertes Wissen zurückgreifen muss.
chain.invoke("Wie geht es dir?")